In [7]:
import os
import tensorflow as tf
from functools import partial
import matplotlib.pyplot as plt

BATCH_SIZE = 32
IMAGE_SIZE = (1024,1024)

AUTOTUNE = tf.data.experimental.AUTOTUNE

In [10]:
cwd = os.path.join(os.getcwd(), "..")
cwd

'/..'

In [ ]:
TRAINING_FILENAMES = tf.io.gfile.glob("/content/melanoma-classification/data/train*.tfrec")
TEST_FILENAMES = tf.io.gfile.glob("/content/melanoma-classification/data/test*.tfrec")

print("Train TFRecord Files:", len(TRAINING_FILENAMES))
print("Test TFRecord Files:", len(TEST_FILENAMES))